In [168]:
import pandas as pd
import numpy as np
import csv
from datetime import timedelta, datetime
pd.set_option('display.max_colwidth', -1)

In [116]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [119]:
from statistic import Statistic
from utils import UtilsKy

In [118]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
# ky9
#df = pd.read_csv(UtilsKy.PATH_KY9 + '02_pure_data/ky9_for_resolve.csv', dtype=str)

In [123]:
# kyw3
df = pd.read_csv(UtilsKy.PATH_KYW3 + '02_pure_data/kyw3_for_resolve.csv', dtype=str)

In [124]:
df.shape

(574426, 24)

In [125]:
Statistic.get_table(df, 'status')

Counter({'false': 565565, 'true': 8861})

In [126]:
df.head()

,order_id,type,status,date,processed,amount,currency,card_holder,email,phone,...,ip,project_id,reported,status_chargeback,date_cb_only,card_masked,card_expire,amount_in_currency,date_only,id
0,49219577955111383,charge,false,2019-11-20 09:39:36,2019-11-20 09:39:41,1.00,USD,Gabriele Markes,gabriele@markes.com,5556789364528,...,84.56.163.85,1a92804aa92f2a762361fcd2b6cc0794360b23bd,NaN,refunded,2019-11-20,547087******1680,2022-04,1.00,2019-11-20,547087******1680_2022-04
1,49221760925063337,charge,false,2019-11-20 12:01:43,2019-11-20 12:01:50,25.00,USD,Malia Gusman,560100032720@4null.com,8086510074,...,204.210.126.255,1a92804aa92f2a762361fcd2b6cc0794360b23bd,NaN,charged,2019-11-20,517148******7320,2023-04,25.00,2019-11-20,517148******7320_2023-04
2,49224688788195598,charge,false,2019-11-20 15:12:20,2019-11-20 15:12:24,21.18,USD,Stephen Gordy,560536900411@4null.com,7406005440,...,99.203.145.83,1a92804aa92f2a762361fcd2b6cc0794360b23bd,NaN,charged,2019-11-20,546540******9404,2023-07,21.18,2019-11-20,546540******9404_2023-07
3,49224696921328680,charge,false,2019-11-20 15:12:52,2019-11-20 15:12:55,200.00,USD,Nicholas Saephanh,560131190505@4null.com,5102929855,...,69.181.49.246,9ffa75299b6caaa89457fb07db86f47f8fb8198a,NaN,charged,2019-11-20,510805******1186,2023-03,200.00,2019-11-20,510805******1186_2023-03
4,49224839756465248,charge,false,2019-11-20 15:22:10,2019-11-20 15:22:13,2.38,USD,Thiago Fernandes,550322045766@4null.com,21994891341,...,179.158.244.208,1a92804aa92f2a762361fcd2b6cc0794360b23bd,NaN,charged,2019-11-20,544731******7316,2026-07,2.38,2019-11-20,544731******7316_2026-07


In [127]:
df.dtypes

order_id              object
type                  object
status                object
date                  object
processed             object
amount                object
currency              object
card_holder           object
email                 object
phone                 object
address               object
city_from_order       object
country               object
zip                   object
ip                    object
project_id            object
reported              object
status_chargeback     object
date_cb_only          object
card_masked           object
card_expire           object
amount_in_currency    object
date_only             object
id                    object
dtype: object

In [128]:
min(df.date_only)

'2019-11-20'

In [129]:
max(df.date_only)

'2020-05-26'

In [130]:
df.date_only = pd.to_datetime(df.date_only)

In [131]:
df.date_cb_only = pd.to_datetime(df.date_cb_only)

In [132]:
start_date = min(df.date_only)
start_date

Timestamp('2019-11-20 00:00:00')

In [133]:
end_date = max(df.date_only)
end_date

Timestamp('2020-05-26 00:00:00')

In [134]:
diff = end_date - start_date
diff_days = diff.days
diff_days

188

In [135]:
days_in_test = int(diff_days * 0.2)
days_in_test = 50
start_test = end_date - timedelta(days=days_in_test)
start_test

Timestamp('2020-04-06 00:00:00')

In [136]:
int(diff_days * 0.025)

4

In [137]:
retreat_days_from_end = int(diff_days * 0.025)
retreat_days_from_end = 20
end_test = end_date - timedelta(days=retreat_days_from_end)
end_test

Timestamp('2020-05-06 00:00:00')

In [138]:
# retreat_days_from_start = 20
# start_teach = start_date + timedelta(days=retreat_days_from_start)
start_teach = start_date
start_teach

Timestamp('2019-11-20 00:00:00')

In [139]:
retreat_days_from_test_start = int(diff_days * 0.025)
retreat_days_from_test_start = 24
end_teach = start_test - timedelta(days=retreat_days_from_test_start)
end_teach

Timestamp('2020-03-13 00:00:00')

In [140]:
start_teach

Timestamp('2019-11-20 00:00:00')

In [141]:
end_teach

Timestamp('2020-03-13 00:00:00')

In [142]:
start_test

Timestamp('2020-04-06 00:00:00')

In [143]:
end_test

Timestamp('2020-05-06 00:00:00')

In [144]:
# если не устраивают даты start_teach, end_teach, start_test, end_test их можно перезадать явно( пример внизу)
# start_teach = datetime.strptime("2020-01-01", '%Y-%m-%d')

In [145]:
good_statuses = Statistic.get_dt_good(df).status.unique()
good_status = good_statuses[0] 
good_status

'false'

In [146]:
mask_teach = (df.date_only < start_test) & ((df.status.isin(Statistic.BAD_STATUSES)) | (df.date_only < end_teach ))
teach = df[mask_teach].copy()

In [147]:
Statistic.get_table(teach, 'status')

Counter({'false': 426222, 'true': 8092})

In [148]:
teach.shape[0]

434314

In [149]:
teach.status = np.where( (teach.status.isin(Statistic.BAD_STATUSES)) & (teach.date_cb_only > start_test), good_status, teach.status)

In [150]:
Statistic.get_table(teach, 'status')

Counter({'false': 427803, 'true': 6511})

In [151]:
mask_test = (df.date_only > start_test) & ((df.status.isin(Statistic.BAD_STATUSES)) | (df.date_only < end_test))
test = df[mask_test]

In [152]:
Statistic.get_table(test, 'status')

Counter({'false': 58107, 'true': 755})

In [153]:
Statistic.get_cb_rate(teach)

1.5

In [154]:
Statistic.get_cb_rate(test)

1.28

In [155]:
Statistic.get_cb_rate_amount(teach)

2.39

In [157]:
Statistic.get_cb_rate_amount(test)

2.41

In [158]:
teach_name = 'teach_' + start_teach.strftime('%Y-%m-%d') + '_' + end_teach.strftime('%Y-%m-%d') + '.csv'
test_name = 'test_' + start_test.strftime('%Y-%m-%d') + '_' + end_test.strftime('%Y-%m-%d') + '.csv'
teach_name

'teach_2019-11-20_2020-03-13.csv'

In [159]:
test_name

'test_2020-04-06_2020-05-06.csv'

In [160]:
path_exp = UtilsKy.PATH_KYW3 + '04_experiments/ex_02_jupyter/'
path_teach = path_exp + teach_name
path_test = path_exp + test_name

In [161]:
drop_columns = ['type', 'amount_in_currency']

In [162]:
teach = teach.drop(columns=drop_columns)
test = test.drop(columns=drop_columns)

In [163]:
list(teach)

['order_id',
 'status',
 'date',
 'processed',
 'amount',
 'currency',
 'card_holder',
 'email',
 'phone',
 'address',
 'city_from_order',
 'country',
 'zip',
 'ip',
 'project_id',
 'reported',
 'status_chargeback',
 'date_cb_only',
 'card_masked',
 'card_expire',
 'date_only',
 'id']

In [170]:
# teach.to_csv(path_teach, index=False, quoting=csv.QUOTE_ALL)
# test.to_csv(path_test, index=False, quoting=csv.QUOTE_ALL)